In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

2023-06-20 21:51:58.904123: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
!ls

Classification_Metrics.md
CountVectorizer.ipynb
Disaster-Tweets-with-TF-Keras-RNN.ipynb
Disaster-Tweets-with-TF-Keras-and-Sklearn.ipynb
Disaster_Tweets_with_TensorFlow.ipynb
NLP.md
Untitled.ipynb
nlp-getting-started
train_test_split.ipynb


In [3]:
!ls nlp-getting-started/

nlp-getting-started.zip test.csv
sample_submission.csv   train.csv


In [4]:
train_csv_df = pd.read_csv("nlp-getting-started/train.csv")

In [5]:
train_df, val_df = train_test_split(train_csv_df, train_size=0.85, random_state=1)

## Naive Approach-- CountVectorizer without Preprocessing

In [6]:
vectorizer = CountVectorizer()
train_bow = vectorizer.fit_transform(train_df["text"])
val_bow = vectorizer.transform(val_df["text"])

In [7]:
train_bow.shape, val_bow.shape

((6471, 19365), (1142, 19365))

In [ ]:
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=19365, output_dim=8),
    tf.keras.layers.SimpleRNN(units=32),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.BinaryCrossentropy(),
                         tf.keras.metrics.BinaryAccuracy(),
                         tf.keras.metrics.AUC()])

model_1.fit(x=train_bow.todense(), y=train_df["target"],
            validation_data=(val_bow.todense(), val_df["target"]),
            epochs=1)

  1/203 [..............................] - ETA: 1:27:27 - loss: 0.6952 - binary_crossentropy: 0.6952 - binary_accuracy: 0.5000 - auc: 0.3828

As we can see, this approach takes too long to be practical without taking other steps.

## Removing Infrequent Words

In [33]:
vectorizer_2 = CountVectorizer(min_df=5, ngram_range=(1,3))
train_bow_2 = vectorizer_2.fit_transform(train_df["text"])
val_bow_2 = vectorizer_2.transform(val_df["text"])

train_bow_2.shape, val_bow_2.shape

((6471, 5389), (1142, 5389))

In [34]:
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=5389, output_dim=32),
    tf.keras.layers.SimpleRNN(units=32),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.BinaryCrossentropy(),
                         tf.keras.metrics.BinaryAccuracy(),
                         tf.keras.metrics.AUC()])

model_1.fit(x=train_bow.todense(), y=train_df["target"],
            validation_data=(val_bow.todense(), val_df["target"]),
            epochs=1)

  1/203 [..............................] - ETA: 1:28:14 - loss: 0.6505 - binary_crossentropy: 0.6505 - binary_accuracy: 0.7500 - auc_3: 0.4583

KeyboardInterrupt: 

Still way too long